In [16]:
:dep ndarray
:dep once_cell
:dep ndarray-rand
:dep rand_isaac
:dep ndarray-parallel
// :dep rayon

use ndarray::prelude::*;
// use ndarray::{concatenate, Axis};
use once_cell::sync::Lazy;
use ndarray_rand::RandomExt;
use ndarray_rand::rand_distr::Uniform;
use rand_isaac::isaac64::Isaac64Rng;
use ndarray_rand::rand::SeedableRng;
// use ndarray::parallel::prelude::*;
use ndarray_parallel::prelude::*;
use ndarray::SliceInfo;
use std::any::Any;

In [17]:
const L_SIZE: usize = 3;

static UP_ROLL_SLC: Lazy<[usize; L_SIZE]> = Lazy::new(|| {  // left
    let mut arr = [0_usize; L_SIZE];
    for i in 0..L_SIZE - 1 {
        arr[i] = i + 1;
    }
    arr[L_SIZE - 1] = 0;
    arr
});
static DOWN_ROLL_SLC: Lazy<[usize; L_SIZE]> = Lazy::new(|| {  // right
    let mut arr = [0_usize; L_SIZE];
    for i in 1..L_SIZE {
        arr[i] = i - 1;
    }
    arr[0] = L_SIZE - 1;
    arr
});

enum Direction {
    Up,
    Down,
    Left,
    Right,
}

fn roll(arr: &Array<f64, Ix2>, direction: Direction) -> Array<f64, Ix2> {
    match direction{
        Direction::Up => {
            concatenate![Axis(0), arr.slice(s![1.., ..]), arr.slice(s![0..1, ..])]
        },
        Direction::Down => {
            concatenate![Axis(0), arr.slice(s![L_SIZE-1.., ..]), arr.slice(s![..L_SIZE-1, ..])]
        },
        Direction::Left => {
            concatenate![Axis(1), arr.slice(s![.., 1..]), arr.slice(s![.., 0..1])]
        },
        Direction::Right => {
            concatenate![Axis(1), arr.slice(s![.., L_SIZE-1..]), arr.slice(s![.., ..L_SIZE-1])]
        },
    }
}

fn nabla2C(c: &Array<f64, Ix2>) -> Array<f64, Ix2> {
    (
        roll(c, Direction::Up) + 
        roll(c, Direction::Down) + 
        roll(c, Direction::Left) + 
        roll(c, Direction::Right) - 
        4. * c
    ) / 4.
}

fn nablaC(c: &Array<f64, Ix2>) -> Array<f64, Ix3> {
    concatenate![
        Axis(2), 
        roll(c, Direction::Right).insert_axis(Axis(2)) - roll(c, Direction::Left).insert_axis(Axis(2)),
        roll(c, Direction::Down).insert_axis(Axis(2)) - roll(c, Direction::Up).insert_axis(Axis(2))
    ]
}

fn cartesian_product(arr1: &Array<f64, Ix1>, arr2: &Array<f64, Ix1>) -> Array2<f64> {
    let (n, m) = (arr1.len(), arr2.len());
    let mut product = Array2::<f64>::zeros((n * m, 2));

    for (i, val1) in arr1.iter().enumerate() {
        for (j, val2) in arr2.iter().enumerate() {
            product[[i * m + j, 0]] = *val1;
            product[[i * m + j, 1]] = *val2;
        }
    }

    product
}

fn _distance_x(deltaX: &Array<f64, Ix3>) -> Array<f64, Ix2> {
    (deltaX.mapv(|x| x.powi(2)).sum_axis(Axis(2))).mapv(f64::sqrt)
}

fn argmin_last(x: &Array<f64, Ix1>) -> usize {
    x.iter()
        .enumerate()
        .scan(None, |state, (i, &val)| {
            match state.as_ref() {
                None => {
                    *state = Some((i, val));
                }
                Some(&(_, min_val)) => {
                    if val <= min_val {
                        *state = Some((i, val));
                    }
                }
            }
            state.map(|s| s.clone())
        })
        .last()
        .unwrap()
        .0
}

fn argmin(arr: &Array<f64, Ix2>) -> Array<usize, Ix1> {
    arr.map_axis(Axis(1), |x| argmin_last(&x.to_owned()))
}

#[derive(Clone)]
pub struct PatternFormationConfig {
    couplingK: f64, alpha: f64, u0: f64, v0: f64, dt: f64, boundaryLength: f64,
    decayRateKd: f64, feedRateKf: f64, productRateKU: f64, productRateKV: f64,
    diffusionRateDU: f64, diffusionRateDV: f64, chemoBetaU: f64, chemoBetaV: f64, 
    agentsNum: usize, cellNumInLine: usize, speedV: f64, randomSeed: u64
}

impl Default for PatternFormationConfig {
    fn default() -> Self {
        PatternFormationConfig{
            couplingK: 1.,
            alpha: 1.,
            u0: 0.012,
            v0: 0.012,
            dt: 0.02,
            boundaryLength: 10.,
            decayRateKd: 1.,
            feedRateKf: 1.,
            productRateKU: 1.,
            productRateKV: 1.,
            diffusionRateDU: 0.01,
            diffusionRateDV: 0.01,
            chemoBetaU: 1.,
            chemoBetaV: 1.,
            agentsNum: 1000,
            cellNumInLine: 250,
            speedV: 3.,
            randomSeed: 10,
        }
    }
}
impl std::fmt::Debug for PatternFormationConfig {
    fn fmt(&self, f: &mut std::fmt::Formatter<'_>) -> std::fmt::Result {
        f.debug_struct("PatternFormationConfig")
            .field("couplingK", &self.couplingK)
            .field("alpha", &self.alpha)
            .field("u0", &self.u0)
            .field("v0", &self.v0)
            .field("dt", &self.dt)
            .field("boundaryLength", &self.boundaryLength)
            .field("decayRateKd", &self.decayRateKd)
            .field("feedRateKf", &self.feedRateKf)
            .field("productRateKU", &self.productRateKU)
            .field("productRateKV", &self.productRateKV)
            .field("diffusionRateDU", &self.diffusionRateDU)
            .field("diffusionRateDV", &self.diffusionRateDV)
            .field("chemoBetaU", &self.chemoBetaU)
            .field("chemoBetaV", &self.chemoBetaV)
            .field("agentsNum", &self.agentsNum)
            .field("cellNumInLine", &self.cellNumInLine)
            .field("speedV", &self.speedV)
            .field("randomSeed", &self.randomSeed)
            .finish()
    }
}

#[derive(Clone)]
pub struct PatternFormation {
    positionX: Array<f64, Ix2>, 
    phaseTheta: Array<f64, Ix1>, freqOmega: Array<f64, Ix1>,
    cemoU: Array<f64, Ix2>, cemoV: Array<f64, Ix2>,
    boundaryLength: f64,
    halfBoundaryLength: f64,
    cellNumInLine: usize,
    cemoPositon: Array<f64, Ix2>,
    dx: f64,
    agentsNum: usize,
    productRateKU: f64,
    productRateKV: f64,
    decayRateKd: f64,
    feedRateKf: f64,
    diffusionRateDU: f64,
    diffusionRateDV: f64,
    chemoBetaU: f64,
    chemoBetaV: f64,
    u0: f64,
    v0: f64,
    dt: f64,
    speedV: f64,
    couplingK: f64,
    alpha: f64,
    config: PatternFormationConfig,
    // tempState
    newAxisPositionX: Array<f64, Ix3>,
    newAxisPhaseTheta: Array<f64, Ix2>,
    tempPositionIdx: Vec<usize>,
    tempDirectionP: Array<f64, Ix2>,
}


impl PatternFormation {
    fn new(config: PatternFormationConfig) -> Self {
        let mut rng = Isaac64Rng::seed_from_u64(config.randomSeed);
        let positionX = Array::random_using(
            (config.agentsNum, 2), 
            Uniform::new(0., config.boundaryLength), 
            &mut rng
        );
        let phaseTheta = Array::random_using(
            config.agentsNum, Uniform::new(0., 2. * std::f64::consts::PI), &mut rng);
        let halfFreq = Array::random_using(config.agentsNum / 2, Uniform::new(1., 3.), &mut rng);
        let freqOmega = concatenate![
            Axis(0), halfFreq.clone(), -halfFreq.clone()
        ];
        let cemoU = Array::ones((config.cellNumInLine, config.cellNumInLine)) * config.u0;
        let cemoV = Array::ones((config.cellNumInLine, config.cellNumInLine)) * config.v0;
        let cemoPositon = cartesian_product(
            &Array::linspace(0., config.boundaryLength, config.cellNumInLine),
            &Array::linspace(0., config.boundaryLength, config.cellNumInLine)
        );
        let dx = config.boundaryLength / (config.cellNumInLine - 1) as f64;
        let halfBoundaryLength = config.boundaryLength / 2.;
        let newAxisPositionX = positionX.clone().insert_axis(Axis(1));
        let newAxisPhaseTheta = phaseTheta.clone().insert_axis(Axis(1));
        let tempPositionIdx = vec![0; config.agentsNum];
        let tempDirectionP = concatenate![
            Axis(1), 
            phaseTheta.mapv(f64::cos).slice(s![.., NewAxis]), 
            phaseTheta.mapv(f64::sin).slice(s![.., NewAxis])
        ];

        PatternFormation {
            positionX: positionX, 
            phaseTheta: phaseTheta, 
            freqOmega: freqOmega,
            cemoU: cemoU, 
            cemoV: cemoV,
            boundaryLength: config.boundaryLength,
            halfBoundaryLength: halfBoundaryLength,
            cellNumInLine: config.cellNumInLine,
            cemoPositon: cemoPositon,
            dx: dx,
            agentsNum: config.agentsNum,
            productRateKU: config.productRateKU,
            productRateKV: config.productRateKV,
            decayRateKd: config.decayRateKd,
            feedRateKf: config.feedRateKf,
            diffusionRateDU: config.diffusionRateDU,
            diffusionRateDV: config.diffusionRateDV,
            chemoBetaU: config.chemoBetaU,
            chemoBetaV: config.chemoBetaV,
            u0: config.u0,
            v0: config.v0,
            dt: config.dt,
            speedV: config.speedV,
            couplingK: config.couplingK,
            alpha: config.alpha,
            config: config,
            newAxisPositionX: newAxisPositionX,
            newAxisPhaseTheta: newAxisPhaseTheta,
            tempPositionIdx: tempPositionIdx,
            tempDirectionP: tempDirectionP,
        }
    }
    fn _delta_x(&self, position: &Array<f64, Ix2>, others: &Array<f64, Ix3>) -> Array<f64, Ix3> {
        // let subX = position - others;
        (position - others).par_mapv_inplace(|x| {
            if x > self.halfBoundaryLength {
                x - self.boundaryLength
            } else if x < -self.halfBoundaryLength {
                x + self.boundaryLength
            } else {
                x
            }
        })
    } 
    fn _mod_position(&self, position: &Array<f64, Ix2>) -> Array<f64, Ix2> {
        position.mapv(|x| {
            if x > self.boundaryLength {
                x - self.boundaryLength
            } else if x < 0. {
                x + self.boundaryLength
            } else {
                x
            }
        })
    }
    fn directionP(&self) -> Array<f64, Ix2> {
        concatenate![
            Axis(1), 
            self.phaseTheta.mapv(f64::cos).slice(s![.., NewAxis]), 
            self.phaseTheta.mapv(f64::sin).slice(s![.., NewAxis])
        ]
    }
    fn deltaCX(&self) -> Array<f64, Ix3> {
        self._delta_x(&self.cemoPositon, &self.newAxisPositionX)
    }   
    fn deltaX(&self) -> Array<f64, Ix3> {
        self._delta_x(&self.positionX, &self.newAxisPositionX)
    }
    fn adjacency(&self) -> Array<f64, Ix2> {
        (_distance_x(&self.deltaX()) / -self.alpha).mapv(|x| f64::exp(x))
    }
    fn nablaU(&self) -> Array<f64, Ix3> {
        nablaC(&self.cemoU) / self.dx
    }
    fn nablaV(&self) -> Array<f64, Ix3> {
        nablaC(&self.cemoV) / self.dx
    }
    fn chemotacticU(&self) -> Array<f64, Ix1> {
        let localGradU = (
            self.nablaU().to_shape((self.cellNumInLine * self.cellNumInLine, 2)).unwrap()
            .select(Axis(0), (&self.tempPositionIdx).as_slice())
        );
        self.chemoBetaU * (
            &self.tempDirectionP.slice(s![.., 0]) * &localGradU.slice(s![.., 1]) - 
            &self.tempDirectionP.slice(s![.., 1]) * &localGradU.slice(s![.., 0])
        )
    }
    fn chemotacticV(&self) -> Array<f64, Ix1> {
        let localGradV = (
            self.nablaV().to_shape((self.cellNumInLine * self.cellNumInLine, 2)).unwrap()
            .select(Axis(0), (&self.tempPositionIdx).as_slice())
        );
        self.chemoBetaV * (
            &self.tempDirectionP.slice(s![.., 0]) * &localGradV.slice(s![.., 1]) - 
            &self.tempDirectionP.slice(s![.., 1]) * &localGradV.slice(s![.., 0])
        )
    }
    fn dotTheta(&self) -> Array<f64, Ix1> {
        &self.freqOmega + &self.chemotacticU() + &self.chemotacticV() 
        + self.couplingK * (
            self.adjacency() * (&self.newAxisPhaseTheta - &self.phaseTheta)
        ).sum_axis(Axis(0))
    }
    fn nabla2U(&self) -> Array<f64, Ix2> {
        nabla2C(&self.cemoU).mapv(|x| x / self.dx.powi(2))
    }
    fn nabla2V(&self) -> Array<f64, Ix2> {
        nabla2C(&self.cemoV).mapv(|x| x / self.dx.powi(2))
    }
    fn update(&mut self) {
        self.newAxisPositionX.assign(&self.positionX.clone().insert_axis(Axis(1)));
        self.newAxisPhaseTheta.assign(&self.phaseTheta.clone().insert_axis(Axis(1)));
        let idxs = (&self.positionX / self.dx).round().mapv(|x| x as usize);
        self.tempPositionIdx = (
            (&idxs.slice(s![.., 0]) * self.cellNumInLine + &idxs.slice(s![.., 1])).as_slice().unwrap().to_vec()
        );
        self.tempDirectionP.assign(&self.directionP());
        self.positionX.assign(&(self.dt * self.speedV * &self.tempDirectionP));
        self.positionX.assign(&self._mod_position(&self.positionX));
    }
}
impl std::fmt::Debug for PatternFormation {
    fn fmt(&self, f: &mut std::fmt::Formatter<'_>) -> std::fmt::Result {
        f.debug_struct("PatternFormation on")
            .field("", &self.config)
            .finish()
    }
}


// let c: Array<f64, Ix2> = Array::zeros((L_SIZE, L_SIZE));
let c: Array<f64, Ix2> = Array::from_shape_vec((3, 3), vec![1., 2., 3., 4., 5., 6., 7., 8., 9.]).unwrap();

Error: cannot find macro `concatenate` in this scope

Error: cannot find macro `concatenate` in this scope

Error: cannot find macro `concatenate` in this scope

Error: cannot find macro `concatenate` in this scope

Error: cannot find macro `concatenate` in this scope

...

Error: cannot find macro `concatenate` in this scope

Error: unnecessary parentheses around assigned value

Error: unnecessary parentheses around assigned value

Error: unnecessary parentheses around assigned value

Error: no method named `par_mapv_inplace` found for struct `ndarray::ArrayBase` in the current scope

In [8]:
let mut model = PatternFormation::new(PatternFormationConfig {
    cellNumInLine: 250,
    ..Default::default() // use default value
});

In [9]:
model.dotTheta();

In [16]:
use std::time::{Instant};
let counts: usize = 100;
fn func(model: &PatternFormation) {
    &model.deltaX();
}

let start = Instant::now();
func(&model);
let mut duration = start.elapsed();
for _ in 0..(counts - 1) {
    let start = Instant::now();
    func(&model);
    duration += start.elapsed();
}
println!("mean duration: {:?}", duration / counts as u32);

mean duration: 19.545639ms
